In [147]:
import pandas as pd
from pyarrow.parquet import ParquetFile
import dask.dataframe as dd
import os
import xarray as xr


In [148]:
dir_files = [os.path.join("short_range_18files", files) for files in os.listdir("short_range_18files")]
print(dir_files)

['short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f001.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f002.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f003.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f004.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f005.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f006.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f007.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f008.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f009.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nwm.t00z.short_range.channel_rt.f010.conus.nc', 'short_range_18files\\nwm.20220911_short_range_nw

In [149]:
fds = []
for f in dir_files:
    fds.append(xr.open_dataset(f))

In [ ]:
%%time
ds = xr.concat(fds,dim="time")
ds

In [ ]:
#vec = ds.feature_id[179] # 1 feature_id
vec = ds.feature_id[1:101]

ds = ds.loc[{"feature_id": vec}]

df = ds.to_dataframe()
df = df.streamflow
ds

In [ ]:
#%time ds.loc[{"feature_id": vec}].plot.scatter("time", "streamflow")

In [ ]:
%%time
df = pd.Series.to_frame(df)
df.to_parquet("data/parquet_100_featureId.gzip"
             , engine="fastparquet" #"pyarrow", "auto"
             , compression="gzip" # "snappy"
            )


In [ ]:
ParquetFile("data/parquet_100_featureId.gzip").metadata #  num_columns: 3

In [ ]:
%%time
data = dd.read_parquet(
    "data/parquet_100_featureId.gzip"
    , storage_options={"anon": True}
    )
data
#%time data.loc[slice(None), int('101')]

#result = data.shape[0].compute()
result = data.compute()
result

r_xa = result.to_xarray()
r_xa
r_xa.plot.scatter("time","streamflow")
